# About: VMの作成

libvirtがインストールされている仮想化基盤上で、VMを作成するためのNotebook。

すでに**VMイメージ作成Notebook**により、イメージが作成されているものとする。

## Operation Note

*ここに経緯を記述*

# Notebookと環境のBinding

Inventory中のgroup名でBind対象を指示する。

In [1]:
target_group = 'test-hypervisor'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


対象マシンにlibvirtがインストールされているかを確認する。

In [3]:
!ansible -b -a 'virsh version' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Compiled against library: libvirt 0.10.2
Using library: libvirt 0.10.2
Using API: QEMU 0.10.2
Running hypervisor: QEMU 0.12.1



# VMイメージの指定

作成対象のVMのあるディレクトリを指定する。**VMイメージ作成Notebook**により生成されたイメージが格納されているディレクトリを指定すること。

In [4]:
image_base_dir = '/mnt/ubuntu14.04-base-vm'

以下の2つのファイルが存在している必要がある。

- base.img
- libvirt-base.xml

In [5]:
!ansible -b -a "ls -la {image_base_dir}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
total 3853356
drwxr-xr-x 2 root root         4096 Jun 17 19:36 .
drwxr-xr-x 6 root root         4096 Jun 17 19:16 ..
-rwxr-xr-x 1 root root 107374182400 Jun 17 19:36 base.img
-rw-r--r-- 1 root root         2461 Jun 17 19:36 libvirt-base.xml



作成するVM名のリストを指定する。お手本では例として2つのVMを指定している。

起動したいVM名をlistで指定すること。**既存のVMと重複してはならない。**

In [6]:
vm_names = ['testvm-001']
vm_names

['testvm-001']

# VMの作成

VM用のファイルは以下のように作成される。

- /mnt
  - (VM名).xml ... libvirtに与えるXML定義
  - (VM名).img ... VM用の仮想ディスク

## XML定義の生成

基本となるXML定義を得る。

In [7]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmp1c6xt5'

In [8]:
!ansible -b -m fetch -a 'src={image_base_dir}/libvirt-base.xml dest={work_dir}/libvirt-base.xml flat=yes' {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "checksum": "544dfe399f5626ed3b316b9d3b0d3041fbc0b922", 
    "dest": "/tmp/tmp1c6xt5/libvirt-base.xml", 
    "md5sum": "6d84ff7f9786c472caa07f0ecdd3d74e", 
    "remote_checksum": "544dfe399f5626ed3b316b9d3b0d3041fbc0b922", 
    "remote_md5sum": null
}


基本のXML定義に基づいて、VM用定義を生成する。

In [9]:
import xml.etree.ElementTree as ET
import virtinst.util
import os

for n in vm_names:
    vmxml = ET.parse(os.path.join(work_dir, 'libvirt-base.xml')).getroot()
    vmxml.find('name').text = n
    vmxml.find('devices').find('disk').find('source').attrib['file'] = os.path.join('/mnt', n + '.img')
    vmxml.find('devices').find('interface').find('mac').attrib['address'] = virtinst.util.randomMAC()
    ET.ElementTree(vmxml).write(os.path.join(work_dir, n + '.xml'))
!ls -la {work_dir}/*.xml

-rw-r--r-- 1 root root 2461 Jun 17 22:36 /tmp/tmp1c6xt5/libvirt-base.xml
-rw-r--r-- 1 root root 2512 Jun 17 22:36 /tmp/tmp1c6xt5/testvm-001.xml


ホストに定義ファイルをコピーする。

In [10]:
for n in vm_names:
    !ansible -b -m copy -a 'src={work_dir}/{n}.xml dest=/mnt/{n}.xml' {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "checksum": "aff0775772be017583b5155fe38993ea0a46cd9f", 
    "dest": "/mnt/testvm-001.xml", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "01081a332cce845696c84fd1d12fde21", 
    "mode": "0644", 
    "owner": "root", 
    "size": 2512, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1466170607.85-174997417957451/source", 
    "state": "file", 
    "uid": 0
}


## イメージファイルのコピー

イメージファイルをVM用に複製する。

In [11]:
for n in vm_names:
    !ansible -b -a 'cp {image_base_dir}/base.img /mnt/{n}.img' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>




## VMの起動

XMLファイル、仮想ディスクファイルがあるかどうかを確認する。

In [12]:
for n in vm_names:
    !ansible -a 'ls -la /mnt/{n}.img /mnt/{n}.xml' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
-rwxr-xr-x 1 root root 107374182400 Jun 17 22:38 /mnt/testvm-001.img
-rw-r--r-- 1 root root         2512 Jun 17 22:36 /mnt/testvm-001.xml



VMを起動する。

In [13]:
import time
for n in vm_names:
    !ansible -b -a 'virsh create /mnt/{n}.xml' {target_group}
    time.sleep(60)

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Domain testvm-001 created from /mnt/testvm-001.xml



VMに設定されたIPアドレスを確認する。

In [14]:
import re
def get_mac_address(vmname):
    domiflist_stdio = !ansible -b -a "virsh domiflist {vmname}" {target_group}
    mac_pattern = re.compile(r'.*bridge.*\s([0-9a-f\:]+)\s*')
    vmmac = [mac_pattern.match(line).group(1) for line in domiflist_stdio if mac_pattern.match(line)][0]
    return vmmac

def get_ip_address(vmmac):
    leases_stdio = !ansible -b -a "grep {vmmac} /var/lib/dnsmasq/dnsmasq.leases" {target_group}
    ip_pattern = re.compile(r'.*\s([0-9a-f\:]+)\s+([0-9\.]+)\s.*')
    ipaddr = [ip_pattern.match(line).group(2) for line in leases_stdio if ip_pattern.match(line)][0]
    return ipaddr

In [15]:
vmdescs = zip(vm_names, map(lambda mac: get_ip_address(mac), map(lambda n: get_mac_address(n), vm_names)))
vmdescs

[('testvm-001', 'XXX.XXX.XXX.66')]

# Inventoryの更新

Inventoryに、作成したマシンのIPアドレスを追加する。変更する前に、現在の内容をコピーしておく。

In [16]:
!cp inventory {work_dir}/inventory-old

[Inventory](../edit/inventory) を修正する。

In [17]:
!diff -ur {work_dir}/inventory-old inventory

--- /tmp/tmp1c6xt5/inventory-old	2016-06-17 22:39:22.008979291 +0900
+++ inventory	2016-06-17 22:40:16.505428369 +0900
@@ -1,2 +1,5 @@
 [test-hypervisor]
 XXX.XXX.XXX.105
+
+[test-vm]
+XXX.XXX.XXX.66


追加したグループ名でpingが通じるかどうかを確認する。

In [18]:
target_vmgroup = 'test-vm'

In [19]:
!ansible -m ping {target_vmgroup}

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


# 後始末

一時ディレクトリを削除する。

In [20]:
!rm -fr {work_dir}